# Imports

In [ ]:
import openmm
from pathlib import Path
from openmm import *
from openmm.app import *
from openmm.unit import *
from openmm.app import PDBFile
from datetime import datetime

In [ ]:
import simtk

In [ ]:
simtk.openmm.version.version

## paths

In [ ]:
input_pdb_path = "../tests/inputs/prepped_receptor_0-processed.pdb"

In [ ]:
# Input Files

pdb = PDBFile(input_pdb_path)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# System Configuration

nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.000001
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
hydrogenMass = 4.0*amu

In [ ]:
# Integration Options

dt = 0.004*picoseconds
temperature = 300*kelvin
friction = 1.0/picosecond
pressure = 1.0*atmospheres
barostatInterval = 25

In [ ]:
steps = 1000000
equilibrationSteps = 1000
platform = Platform.getPlatformByName('Reference')
dcdReporter = DCDReporter('trajectory.dcd', 10000)
dataReporter = StateDataReporter('log.txt', 1000, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 10000)

## original

In [ ]:
# print('Building system...')
# topology = pdb.topology
# positions = pdb.positions
# modeller = Modeller(topology, positions)
# modeller.addSolvent(forcefield, padding=0.9*nanometers, model='tip3p')
# system = forcefield.createSystem(modeller.topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
#     constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)
# system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))
# integrator = LangevinMiddleIntegrator(temperature, friction, dt)
# integrator.setConstraintTolerance(constraintTolerance)
# simulation = Simulation(topology, system, integrator, platform)
# simulation.context.setPositions(modeller.positions)

In [ ]:
# positions = simulation.context.getState(getPositions=True).getPositions()
# PDBFile.writeFile(simulation.topology, positions, open('0step_minimization.pdb', 'w'))

In [ ]:
# len(positions)

In [ ]:
# len(list(simulation.topology.atoms()))

## Improved

In [ ]:
steps = 1000000
equilibrationSteps = 1000
platform = Platform.getPlatformByName('Reference')
dcdReporter = DCDReporter('trajectory.dcd', 10000)
dataReporter = StateDataReporter('log.txt', 1000, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 10000)

In [ ]:
print(datetime.now().isoformat())
print('Building system...')
platform = Platform.getPlatformByName('Reference')
topology = pdb.topology
positions = pdb.positions

print("Modeller")
print(datetime.now().isoformat())
modeller = Modeller(topology, positions)
modeller.addSolvent(forcefield, padding=0.9*nanometers, model='tip3p')

print("System")
print(datetime.now().isoformat())
system = forcefield.createSystem(modeller.topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))

print("Integrator")
print(datetime.now().isoformat())
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)

print("Simulation")
print(datetime.now().isoformat())

simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
print("Done")
print(datetime.now().isoformat())

In [ ]:
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('0step_minimization.pdb', 'w'))

In [ ]:
len(positions)

In [ ]:
len(list(simulation.topology.atoms()))

In [ ]:
len(modeller.positions)

In [ ]:
len(list(modeller.topology.atoms()))

In [ ]:
print(datetime.now().isoformat())
## Run minimization
print(
    "  initial : %8.3f kcal/mol"
    % (
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)
simulation.minimizeEnergy(maxIterations=5)
print(
    "  final : %8.3f kcal/mol"
    % (
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)
print(datetime.now().isoformat())

In [ ]:
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('5step_minimization.pdb', 'w'))

## Improved with new platform

## CPU

In [ ]:
print(datetime.now().isoformat())
print('Building system...')
platform = Platform.getPlatformByName('CPU')
topology = pdb.topology
positions = pdb.positions

print("Modeller")
print(datetime.now().isoformat())
modeller = Modeller(topology, positions)
modeller.addSolvent(forcefield, padding=0.9*nanometers, model='tip3p')

print("System")
print(datetime.now().isoformat())
system = forcefield.createSystem(modeller.topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))

print("Integrator")
print(datetime.now().isoformat())
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)

print("Simulation")
print(datetime.now().isoformat())

simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
print("Done")
print(datetime.now().isoformat())

In [ ]:
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('0step_minimization_cpu.pdb', 'w'))

In [ ]:
len(positions)

In [ ]:
len(list(simulation.topology.atoms()))

In [ ]:
len(modeller.positions)

In [ ]:
len(list(modeller.topology.atoms()))

In [ ]:
print(datetime.now().isoformat())
## Run minimization
print(
    "  initial : %8.3f kcal/mol"
    % (
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)
simulation.minimizeEnergy()
print(
    "  final : %8.3f kcal/mol"
    % (
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)
print(datetime.now().isoformat())

In [ ]:
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('5step_minimization_opencl.pdb', 'w'))

## OpenCL

In [ ]:
# forcefield = ForceField('amber/ff14SB.xml', 'amber/tip3p_standard.xml')

print(datetime.now().isoformat())
print('Building system...')
platform = Platform.getPlatformByName('OpenCL')
topology = pdb.topology
positions = pdb.positions

print("Modeller")
print(datetime.now().isoformat())
modeller = Modeller(topology, positions)
modeller.addSolvent(forcefield, padding=0.9*nanometers, model='tip3p')

print("System")
print(datetime.now().isoformat())
system = forcefield.createSystem(modeller.topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)

print("Adding Force")
print(datetime.now().isoformat())
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))

print("Integrator")
print(datetime.now().isoformat())
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)

print("Simulation")
print(datetime.now().isoformat())

simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
print("Done")
print(datetime.now().isoformat())